# 01. Dcard 爬蟲

In [2]:
import pandas as pd
import requests
from requests_html import HTML
import re

### 初步檢視抓到的資料結構

In [3]:
import json
ID = '232760457'
url = 'https://www.dcard.tw/_api/posts/' + ID 
requ = requests.get(url)
requ.json()

{'id': 232760457,
 'title': '尋找政大失物！',
 'content': '有人可以幫我尋找我的失物嗎\n我找了很久一直也找不到QaQ\nhttps://i.imgur.com/1SJeHMY.jpg\nhttps://i.imgur.com/FPx4IQx.jpg',
 'excerpt': '有人可以幫我尋找我的失物嗎，我找了很久一直也找不到QaQ',
 'anonymousSchool': False,
 'anonymousDepartment': False,
 'pinned': False,
 'forumId': 'caf9487a-1994-494d-81f2-7950438e7bef',
 'replyId': None,
 'createdAt': '2019-12-25T15:43:01.798Z',
 'updatedAt': '2019-12-25T15:43:49.745Z',
 'commentCount': 16,
 'likeCount': 48,
 'tags': [],
 'topics': ['失物招領', '政大', '聖誕節', '脫魯', '政治大學'],
 'supportedReactions': None,
 'withNickname': False,
 'reportReason': '',
 'hiddenByAuthor': False,
 'meta': {'layout': 'classic'},
 'forumName': '政治大學',
 'forumAlias': 'nccu',
 'school': '國立政治大學',
 'department': '經濟學系',
 'replyTitle': None,
 'gender': 'M',
 'personaSubscriptable': False,
 'reactions': [{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03', 'count': 38},
  {'id': 'e8e6bc5d-41b0-4129-b134-97507523d7ff', 'count': 5},
  {'id': '514c2569-fd53-4d9d-a415-bf0f88e7329f',

### 轉為 DataFrame

In [4]:
rejs = requ.json()
pd.DataFrame(
    data=
    [{'ID':rejs['id'],
      'title':rejs['title'],
      'content':rejs['content'],
      'excerpt':rejs['excerpt'],
      'createdAt':rejs['createdAt'],
      'updatedAt':rejs['updatedAt'],
      'commentCount':rejs['commentCount'],
      'forumName':rejs['forumName'],
      'forumAlias':rejs['forumAlias'],
      'gender':rejs['gender'],
      'likeCount':rejs['likeCount'],
      'reactions':rejs['reactions'],
      'topics':rejs['topics']}],
    columns=['ID','title','content','excerpt','createdAt','updatedAt','commentCount','forumName','forumAlias','gender','likeCount','reactions','topics'])

,ID,title,content,excerpt,createdAt,updatedAt,commentCount,forumName,forumAlias,gender,likeCount,reactions,topics
0,232760457,尋找政大失物！,有人可以幫我尋找我的失物嗎\n我找了很久一直也找不到QaQ\nhttps://i.imgur...,有人可以幫我尋找我的失物嗎，我找了很久一直也找不到QaQ,2019-12-25T15:43:01.798Z,2019-12-25T15:43:49.745Z,16,政治大學,nccu,M,48,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[失物招領, 政大, 聖誕節, 脫魯, 政治大學]"


# 02. 分析PTT八卦板產生一ㄍ文字雲

In [9]:
from ptt import Board
from wordcloud import WordCloud
import matplotlib.pyplot as plt

font_path = '微軟正黑體.ttf'

## 選擇 PTT 八卦板

In [10]:
Board_Name = 'gossiping'
latest_page = Board(Board_Name)

## 爬取文章標題及內文

In [11]:
texts = ''
for summary in latest_page:
    if summary.isremoved:
        continue
    print('爬取文章 -> ' + summary.title)
    article = summary.read()
    texts += article.title + article.content
print('done')

爬取文章 -> [公告] 一劍無悔，十二月置底閒聊文
爬取文章 -> ［協尋］12/23南下國道一火燒車事故行車紀錄器
爬取文章 -> [協尋] 友人妹妹失蹤 代po
爬取文章 -> [協尋] 基隆市七堵區八德路段6號 車禍昏迷
爬取文章 -> [公告] 八卦板板規(2019.08.21)
爬取文章 -> [問卦] ptt帳號最多可以傳到幾代呢？
爬取文章 -> [新聞] 岳母過生日拿不出3000元 他遭悍妻暴踢下體
爬取文章 -> Re: [問卦] 銀行襄理要怎麼把到陳綺貞
爬取文章 -> Re: [新聞] F16V全裝武器亮相 AIM-9X飛彈下月公開展
爬取文章 -> Re: [新聞] 談反滲透法 柯文哲自嘲：會不會選後第一
爬取文章 -> [問卦] 作什麼動作會讓人會心一笑呢？
爬取文章 -> Re: [問卦] 　／｜｜｜＼　PTT斷層效應是不是出現了?
爬取文章 -> Re: [問卦] ～（＠ｏ＠）～ 有沒有已經沒人用電腦看PTT了的八卦？
爬取文章 -> Re: [問卦] 　　／　＼現實社會是不是普男比帥哥更吃香?
爬取文章 -> [新聞] 評施政民調墊底 柯Ｐ：中國官員素質比台
done


## 設定要移除的冗言贅字 (PTT其他看板可能不適用)

In [12]:
removeword = ['span','class','f3','https','imgur','h1','_blank','href','rel',
              'nofollow','target','cdn','cgi','b4','jpg','hl','b1','f5','f4',
              'goo.gl','f2','email','map','f1','f6','__cf___','data','bbs',
              'html','cf','f0','b2','b3','b5','b6','原文內容','原文連結','作者',
              '標題','時間','看板','問卦','新聞','協尋','Re','ltn', 'news', 'tw',
              '視同鬧板','鬧板','註1','註2','刪除','備註','文章','有下列情事者',
              '引述','公告','完整連結','含回文','或短網址','完整內文','退文','違者',
              '記者署名','完整']

for word in removeword:
    texts = texts.replace(word,'')

## 產生文字雲，看看最近在夯什麼！

In [ ]:
wordcloud = WordCloud(font_path=font_path).generate(texts)
plt.figure(figsize=(10,8), facecolor='k' ,frameon=False)

In [ ]:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# 03. 用 Line 即時傳送 PTT 爬蟲資訊
## 匯入所需模組
安裝ptt模組 : pip install ptt.py

In [14]:
from ptt import Board
import requests
import time

### 利用 IFTTT 中的 webhooks 功能，當訪問特定的 url 時，可以連動到 Line 傳送通知。
### 爬蟲的資訊及相關變數則利用 url 傳入參數的方式丟給 webhooks (用requests訪問)

In [15]:
ptt_url = 'https://www.ptt.cc/'
Board_Name = 'rent_ya'
keywords = ['文山區', '政大', '木柵']
app_name = 'ptt_scraper'
token = 'xxxxxxxxxxxxxxx' # 此為個人金鑰
value1 = Board_Name
value2 = ', '.join(keywords)

restart_time = 10
collected = []

while True:
    first_page = Board(Board_Name)
    new_collect = []
    for post in first_page:
        if post.isremoved:
            continue
        post_title = post.title
        post_url = post.url

        for keyword in keywords:
            if keyword in post_title and post_url not in collected:
                collected.append(post_url)
                new_collect.append(post_title)
                new_collect.append(ptt_url + post_url)
                print('抓到了', post_title, ptt_url+post_url, sep='\n')
    
    if new_collect:
        value3 = '<br>'.join(new_collect)
        url = 'https://maker.ifttt.com/trigger/{}/with/key/{}?value1={}&value2={}&value3={}'.format(app_name, token, value1,
                                                                                          value2, value3)
        requests.get(url)
    
    if len(collected) > 50:
        collected.pop()
        
    time.sleep(restart_time)
    

抓到了
[女/台北/文山] 近政大木新路三段----溫馨雅房
https://www.ptt.cc//bbs/Rent_ya/M.1577401012.A.E1B.html


ConnectionError: HTTPSConnectionPool(host='www.ptt.cc', port=443): Max retries exceeded with url: /bbs/rent_ya/index.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1103ab080>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))